# Financial App

In [1]:
import os
import pandas as pd

# Get the current working directory
os.chdir("..")
# Verify the current working directory
print(f"Now in directory: {os.getcwd()}")

# Import your module
from app.modules.database.connection import engine
import app.modules.database.credit_manager as credit_manager
from importlib import reload

%run logs/run_log.py

Now in directory: c:\Users\juanm_8qa8lav\Documents\Proyectos_Personales\FinancialApp


IntegrityError: (pymysql.err.IntegrityError) (1062, "Duplicate entry '1' for key 'provinces.PRIMARY'")
[SQL: INSERT INTO provinces (`ID`, `Name`) VALUES (%(ID)s, %(Name)s)]
[parameters: [{'ID': 1, 'Name': 'Buenos Aires'}, {'ID': 4, 'Name': 'Catamarca'}, {'ID': 5, 'Name': 'Chaco'}, {'ID': 2, 'Name': 'Chubut'}, {'ID': 3, 'Name': 'Ciudad Autónoma de Buenos Aires'}, {'ID': 7, 'Name': 'Corrientes'}, {'ID': 6, 'Name': 'Córdoba'}, {'ID': 8, 'Name': 'Entre Ríos'}  ... displaying 10 of 24 total bound parameter sets ...  {'ID': 23, 'Name': 'Tierra del Fuego'}, {'ID': 24, 'Name': 'Tucumán'}]]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

In [2]:
# reload(credit_manager)
from app.modules.database.credit_manager import credits_balance
balance = credits_balance()
print(f'Deuda Total: $ {balance['Total'].sum():,.2f}')

# balance[['Capital', 'Interest', 'IVA', 'Total']] = balance[['Capital', 'Interest', 'IVA', 'Total']].map('${:,.2f}'.format)
balance.loc[balance['ID_Op'] == 2]

Deuda Total: $ 5,452,862.28


,ID_Op,Nro_Inst,D_Due,Capital,Interest,IVA,Total,ID_Owner
ID,,,,,,,,
7,2,1,2025-01-28,16097.76,66568.55,13979.40,96645.71,1
8,2,2,2025-02-28,18691.05,64425.34,13529.32,96645.71,1
9,2,3,2025-03-28,21702.10,61936.87,13006.74,96645.71,1
10,2,4,2025-04-28,25198.22,59047.51,12399.98,96645.71,1
11,2,5,2025-05-28,29257.54,55692.70,11695.47,96645.71,1
12,2,6,2025-06-28,33970.81,51797.43,10877.46,96645.71,1
13,2,7,2025-07-28,39443.37,47274.66,9927.68,96645.71,1
14,2,8,2025-08-28,45797.54,42023.28,8824.89,96645.71,1
15,2,9,2025-09-28,53175.33,35925.93,7544.45,96645.71,1


In [3]:
reload(credit_manager)
from app.modules.database.credit_manager import TypeDataCollection, read_collection_file, collection_w_early_cancel

amount = balance.loc[balance['ID_Op'] == 2, 'Capital'].sum() - 2000 + 66568.55 + 13979.40 + 64425.34 + 13529.32 + 1000
collections, penalties, installments = collection_w_early_cancel(TypeDataCollection.ID_Op, 2, amount, 3, pd.Timestamp("2025/03/25"), True)

print(f'COMÚN :$ {collections.loc[collections['Type_Collection'] == 'COMÚN', 'Total'].sum():,.2f}')
print(f'CAN. ANT. :$ {collections.loc[collections['Type_Collection'] == 'CAN. ANT.', 'Total'].sum():,.2f}')
print(f'PENALTY :$ {collections.loc[collections['Type_Collection'] == 'PENALTY', 'Total'].sum():,.2f}')
print(f'Total :$ {collections.loc[~collections['Type_Collection'].isin(['BON. CAN. ANT.', 'REDONDEO']), 'Total'].sum():,.2f}')

collections

[2]
Amount: $ 657,502.61
Amount: $ 464,211.19
Amount: $ 82,236.63
Amount: $ 82,236.63

COMÚN :$ 193,291.42
CAN. ANT. :$ 381,974.56
PENALTY :$ 0.00
Total :$ 657,502.61


,ID_Inst,D_Emission,Type_Collection,Capital,Interest,IVA,Total
ID,,,,,,,
0,7,2025-03-25,COMÚN,16097.76,66568.55,13979.40,9.664571e+04
1,8,2025-03-25,COMÚN,18691.05,64425.34,13529.32,9.664571e+04
2,9,2025-03-25,CAN. ANT.,21702.10,0.00,0.00,2.170210e+04
3,10,2025-03-25,CAN. ANT.,25198.22,0.00,0.00,2.519822e+04
4,11,2025-03-25,CAN. ANT.,29257.54,0.00,0.00,2.925754e+04
5,12,2025-03-25,CAN. ANT.,33970.81,0.00,0.00,3.397081e+04
6,13,2025-03-25,CAN. ANT.,39443.37,0.00,0.00,3.944337e+04
7,14,2025-03-25,CAN. ANT.,45797.54,0.00,0.00,4.579754e+04
8,15,2025-03-25,CAN. ANT.,53175.33,0.00,0.00,5.317533e+04


In [4]:
penalties

""


In [5]:
installments

""


In [8]:
from app.modules.database.credit_manager import credits_balance
balance = credits_balance()
print(f'Deuda Total: $ {balance['Total'].sum():,.2f}')

# balance[['Capital', 'Interest', 'IVA', 'Total']] = balance[['Capital', 'Interest', 'IVA', 'Total']].map('${:,.2f}'.format)
balance.loc[balance['ID_Op'].isin([1, 2])]

Deuda Total: $ 5,452,862.28


,ID_Op,Nro_Inst,D_Due,Capital,Interest,IVA,Total,ID_Owner
ID,,,,,,,,
1,1,1,2025-01-28,112940.88,127702.93,26817.62,267461.43,1
2,1,2,2025-02-28,130392.56,113280.05,23788.81,267461.43,1
3,1,3,2025-03-28,150540.89,96628.54,20291.99,267461.43,1
4,1,4,2025-04-28,173802.55,77404.03,16254.85,267461.43,1
5,1,5,2025-05-28,200658.62,55208.93,11593.88,267461.43,1
6,1,6,2025-06-28,231664.50,29584.24,6212.69,267461.43,1
7,2,1,2025-01-28,16097.76,66568.55,13979.40,96645.71,1
8,2,2,2025-02-28,18691.05,64425.34,13529.32,96645.71,1
9,2,3,2025-03-28,21702.10,61936.87,13006.74,96645.71,1


In [ ]:
reload(credit_manager)
from app.modules.database.credit_manager import TypeDataCollection, massive_early_collection


collections, penalties, installments, error  = massive_early_collection('inputs/Cobranza - AMUF.xlsx', 2, TypeDataCollection.DNI, pd.Timestamp("2025/03/25"), save=True)

collections

                  Amount
DNI                     
36329758   $1,605,768.58
45678901  $10,000,000.00
36329758:
45678901:


,ID_Inst,D_Emission,Type_Collection,Capital,Interest,IVA,Total
ID,,,,,,,
0,19,2025-03-25,COMÚN,7982.65,116834.60,24535.27,149352.51
1,20,2025-03-25,COMÚN,9393.28,115668.79,24290.45,149352.51
2,21,2025-03-25,COMÚN,11053.19,114296.96,24002.36,149352.51
3,22,2025-03-25,CAN. ANT.,13006.43,0.00,0.00,13006.43
4,23,2025-03-25,CAN. ANT.,15304.82,0.00,0.00,15304.82
5,24,2025-03-25,CAN. ANT.,18009.37,0.00,0.00,18009.37
6,25,2025-03-25,CAN. ANT.,21191.85,0.00,0.00,21191.85
7,26,2025-03-25,CAN. ANT.,24936.71,0.00,0.00,24936.71
8,27,2025-03-25,CAN. ANT.,29343.34,0.00,0.00,29343.34


In [43]:
penalties

,ID_External,ID_Client,Date_Settlement,ID_BP,Cap_Requested,Cap_Grant,N_Inst,First_Inst_Purch,TEM_W_IVA,V_Inst,First_Inst_Sold,D_F_Due,ID_Purch,ID_Sale
ID,,,,,,,,,,,,,,
0,NaN,1,2025-03-25,1,0.0,0.0,1,NaN,0.0,386140.169,NaN,2025-03-25,NaN,NaN


In [44]:
installments

,ID_Op,Nro_Inst,D_Due,Capital,Interest,IVA,Total,ID_Owner
ID,,,,,,,,
0,10,1,2025-03-25,0.0,319124.11,67016.06,386140.17,1


In [45]:
error

,Amount
DNI,
45678901,10000000.0


In [37]:
balance = credits_balance()
print(f'Deuda Total: $ {balance['Total'].sum():,.2f}')

# balance[['Capital', 'Interest', 'IVA', 'Total']] = balance[['Capital', 'Interest', 'IVA', 'Total']].map('${:,.2f}'.format)
balance

Deuda Total: $ 2,688,345.18


,ID_Op,Nro_Inst,D_Due,Capital,Interest,IVA,Total,ID_Owner
ID,,,,,,,,
1,1,1,2025-01-28,0.00,0.00,0.00,0.00,1
2,1,2,2025-02-28,0.00,0.00,0.00,0.00,1
3,1,3,2025-03-28,0.00,0.00,0.00,0.00,1
4,1,4,2025-04-28,0.00,0.00,0.00,0.00,1
5,1,5,2025-05-28,0.00,0.00,0.00,0.00,1
6,1,6,2025-06-28,0.00,0.00,0.00,0.00,1
7,2,1,2025-01-28,0.00,0.00,0.00,0.00,1
8,2,2,2025-02-28,0.00,0.00,0.00,0.00,1
9,2,3,2025-03-28,0.00,0.00,0.00,0.00,1


In [31]:
pd.read_sql('credits', engine, index_col='ID')

,ID_External,ID_Client,Date_Settlement,ID_BP,Cap_Requested,Cap_Grant,N_Inst,First_Inst_Purch,TEM_W_IVA,V_Inst,First_Inst_Sold,D_F_Due,ID_Purch,ID_Sale
ID,,,,,,,,,,,,,,
1,None,1,2024-11-28,4,1000000.0,1000000.0,6,0.0,0.154521,267461.42,0.0,2025-01-28,None,None
2,None,2,2024-11-28,4,1000000.0,500000.0,12,0.0,0.161096,96645.71,0.0,2025-01-28,None,None
3,None,1,2024-10-15,2,1000000.0,800000.0,18,5.0,0.176712,149352.51,0.0,2024-12-28,None,None
4,None,1,2025-03-25,1,0.0,0.0,1,NaN,0.000000,8410.58,NaN,2025-03-25,None,None
5,None,2,2025-03-25,1,0.0,0.0,1,NaN,0.000000,9341497.39,NaN,2025-03-25,None,None
